In [1]:
import pylibxc
import numpy as np

# util to define variables...

def get_density(r_s):
  return 3/(4*np.pi*(r_s ** 3))

def hartree_to_mRy(energy):
  return energy * 2 * 1000

def get_eps_x_unif(n):
  return -(3/(4*np.pi))*((n*3*np.pi**2)**(1/3))

def get_grad_n(s, n):
  return s*((n**(4/3))*2*(3*np.pi**2)**(1/3))

def get_up_dn_density(n, zeta):
  n = np.expand_dims(n.flatten(), axis=1)
  zeta = np.expand_dims(zeta.flatten(), axis=1)

  up_density = ((zeta * n) + n)/2
  dn_density = (n - (zeta * n))/2
  return np.concatenate((up_density, dn_density), axis=1)

def get_tau(alpha, grad_n, n):
  tau_w = (grad_n**2) / (8*n)
  tau_unif = (3/10)*((3*np.pi)**(2/3))*(n**(5/3))
  
  return alpha*tau_unif + tau_w


In [2]:
def scan_eps_c_pol(r_s, zeta, s, alpha, config):

  scan_c = pylibxc.LibXCFunctional("mgga_c_scan", "polarized")

  n = get_density(r_s)
  grad_n = get_grad_n(s, n)
  n_spin = get_up_dn_density(n, zeta)
  tau = get_tau(alpha, grad_n, n)

  inp = {}
  inp["rho"] = n_spin

  # total sigma and tau
  sigma = np.expand_dims(grad_n.flatten()**2, axis=1)
  tau = np.expand_dims(tau.flatten(), axis=1)

  if config==0:
    sigma = np.concatenate((sigma, np.zeros_like(sigma), np.zeros_like(sigma)), axis=1)
    tau = np.concatenate((tau, np.zeros_like(tau)), axis=1)
  elif config==1:
    sigma = np.concatenate((np.zeros_like(sigma), np.zeros_like(sigma), sigma), axis=1)
    tau = np.concatenate((np.zeros_like(tau),tau), axis=1)
  elif config==2:
    sigma = np.concatenate((sigma/4, sigma/4, sigma/4), axis=1)
    tau = np.concatenate((tau/2, tau/2), axis=1)


  inp["sigma"] = sigma
  inp["tau"] = tau

  scan_c_res = scan_c.compute(inp)
  return np.squeeze(scan_c_res['zk'])


In [3]:
import pandas as pd

r_s = np.array([2])
s = np.array([2])
alpha = np.array([2])
zetas = [0,1,-1]
configs = [0,1,2]

df={zeta:[] for zeta in zetas}
for config in configs:
  for zeta in df:
    df[zeta].append(scan_eps_c_pol(r_s=r_s, zeta=np.array([zeta]), s=s, alpha=alpha, config=config))

df["config"] = configs 
df = pd.DataFrame.from_dict(df)

df


,0,1,-1,config
0,-0.024670419207159677,-0.005251703880224634,-0.03514637668112172,0
1,-0.024670419207159677,-0.03514637668112172,-0.005251703880224634,1
2,-0.018702163852413748,-0.013947485107505538,-0.013947485107505538,2


In [4]:
# values from my mathematica implementation..
# zeta=0 : -0.0194047
# zeta=1,-1 : -0.010152
